In [1]:
import talib as ta
import os
import matplotlib.pyplot as plt
# import mplfinance as mpf
import pandas as pd
import numpy as np
"""abspath = os.path.abspath(__file__)
dname = os.path.dirname(abspath)
os.chdir(dname)"""
data_pairs = pd.read_pickle("data_pairs_kucoin_future_1d.pkl")
pairs = pd.read_pickle("pairs_kucoin_future.pkl")

idex= [*range(0, len(data_pairs), 1)]
ADX_positions  =  pd.DataFrame(index=idex,columns = ['trending_bullish','trending_bearish','best_positions','non_trending'])

WMA_positions  =  pd.DataFrame(index=idex,columns = ['Buy_WMA20','SELL_WMA20','Buy_WMA60',
                                                     'SELL_WMA60','Buy_WMA100','SELL_WMA100','Buy_WMA200','SELL_WMA200'])

MACD_positions =  pd.DataFrame(index=idex,columns = ['trending_bearish_Buy_weak','trending_bullish_sell_weak',
                    'trending_bearish_sell_strong','trending_bullish_Buy_strong','Buy_cross_signal','sell_cross_signal'])

RSI_positions  =  pd.DataFrame(index=idex,columns = ['cross_overbought','cross_oversold'])
BB_positions   =  pd.DataFrame(index=idex,columns = ['Buy_BB','sell_BB'])
OBV_positions  =  pd.DataFrame(index=idex,columns = ['Buy_OBV','sell_OBV'])

class calculate_ta :

            def __init__(self,data_pairs=data_pairs,pairs=pairs):
                self.data_pairs=data_pairs
                self.pairs=pairs

            def ADX(self):
                    print('\n \n ADX ')
                    
                    # creat dataframes for positions


                    for j in range(len(self.data_pairs)):

                            if (type(self.data_pairs.at[j,'pair'])==str) | (len(self.data_pairs.at[j,'pair'])== 0):
                                continue
                            print(j,end="-")
                            test=self.data_pairs.at[j,'pair'].copy()
                    #ADX

                            test['ADX'] = ta.ADX(test['high'], test['low'], test['close'], timeperiod=14)
                            DIplus = ta.PLUS_DI(test['high'], test['low'], test['close'], timeperiod=14)
                            DIminus=ta.MINUS_DI(test['high'], test['low'], test['close'], timeperiod=14)

                    #positions

                            trending_bullish = (test['ADX'].iloc[-1]>20) & (DIplus.iloc[-1] > DIminus.iloc[-1])\
                        & (DIplus.iloc[-2] >= DIminus.iloc[-2])
                        
                            trending_bearish = (test['ADX'].iloc[-1]>20) & (DIplus.iloc[-1] < DIminus.iloc[-1]) \
                            & (DIplus.iloc[-2] <= DIminus.iloc[-2])
                            
                            best_positions = (test['ADX'].iloc[-1]>25) & (test['ADX'].iloc[-1] < DIplus.iloc[-1]) \
                            & (test['ADX'].iloc[-1] >= DIminus.iloc[-1]) | (test['ADX'].iloc[-1]>25) \
                            & (test['ADX'].iloc[-1] > DIplus.iloc[-1]) & (test['ADX'].iloc[-1] < DIminus.iloc[-1])
                            
                            non_trending = test['ADX'].iloc[-1]<20

                            positions_ADX = [trending_bullish,trending_bearish,best_positions,non_trending]

                            n=0
                            for i in positions_ADX:

                                if i==True:
                                    ADX_positions.iat[j,n]=i
                                    n+=1
                                if i==False:
                                    n+=1
                    return ADX_positions

                    #---------------------------------------------------------------------------------------------------------------------#

            def WMA(self):
                    print('\n \n WMA ')
                    for j in range(len(self.data_pairs)):

                            if (type(self.data_pairs.at[j,'pair'])==str) | (len(self.data_pairs.at[j,'pair'])== 0):
                                continue
                            print(j,end="-")
                            test=self.data_pairs.at[j,'pair']
                    #WMA

                            WMA20 = ta.WMA(test['close'], timeperiod=20)
                            WMA60 = ta.WMA(test['close'], timeperiod=60)
                            WMA100 = ta.WMA(test['close'], timeperiod=100)
                            WMA200 = ta.WMA(test['close'], timeperiod=200)


                    #positions

                    # WMA20
                            Buy_WMA20 =  (test['close'] > WMA20) & (test['close'].shift(1) <= WMA20)
                            SELL_WMA20 = (test['close'] < WMA20) & (test['close'].shift(1) >= WMA20)

                    # WMA60
                            Buy_WMA60 =  (test['close'] > WMA60) & (test['close'].shift(1) <= WMA60)
                            SELL_WMA60 = (test['close'] < WMA60) & (test['close'].shift(1) >= WMA60)

                    # WMA100
                            Buy_WMA100 =  (test['close'] > WMA100) & (test['close'].shift(1) <= WMA100)
                            SELL_WMA100 = (test['close'] < WMA100) & (test['close'].shift(1) >= WMA100)

                    # WMA200
                            Buy_WMA200 =  (test['close'] > WMA200) & (test['close'].shift(1) <= WMA200)
                            SELL_WMA200 = (test['close'] < WMA200) & (test['close'].shift(1) >= WMA200)

                    #reset index for idmax
                            list_dataframes = [Buy_WMA20,SELL_WMA20, Buy_WMA60,SELL_WMA60, Buy_WMA100,SELL_WMA100, Buy_WMA200,SELL_WMA200]

                            for i in list_dataframes:
                                i.reset_index(drop=True,inplace=True)

                            positions_WMA = [Buy_WMA20,SELL_WMA20, Buy_WMA60,SELL_WMA60, Buy_WMA100,SELL_WMA100, Buy_WMA200,SELL_WMA200]

                            n=0
                            for i in positions_WMA:
                                g=np.where(i == True)  #find true value that place in a tuple

                                if pd.DataFrame(g).empty==False:
                                    h=(np.max(g)>=list(test.index)[-3]) and (np.max(g)<=list(test.index)[-3])
                                    if h==False:
                                        n+=1
                                    if h==True: # if latest three item wase true we can chose it

                                        WMA_positions.iat[j,n]=np.max(g)
                                        n+=1

                                if pd.DataFrame(g).empty==True:
                                    n+=1
                    return WMA_positions

                    #---------------------------------------------------------------------------------------------------------------------#
                    #MACD   

            def MACD(self):
                    print('\n \n MACD ')
                    for j in range(len(self.data_pairs)):

                            if (type(self.data_pairs.at[j,'pair'])==str) | (len(self.data_pairs.at[j,'pair'])== 0):
                                continue
                            print(j,end="-")
                            test=self.data_pairs.at[j,'pair']

                            EMA12 = ta.EMA(test['close'], timeperiod=12)
                            EMA26 = ta.EMA(test['close'], timeperiod=26)
                            macd, signal, histogram = ta.MACD(test['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)

                            macd = pd.DataFrame(macd)
                            signal = pd.DataFrame(signal)
                            histogram = pd.DataFrame(histogram)

                    #positions

                    #weak signal buy and sell
                            trending_bearish_Buy_weak =  (macd > signal) & (macd.shift(1) <= signal)&(macd <= 0)
                            trending_bullish_sell_weak = (macd < signal) & (macd.shift(1) >= signal)&(macd >= 0)

                    #strong signal buy and sell
                            trending_bearish_sell_strong =  (macd < signal) & (macd.shift(1) >= signal)&(macd <= 0)
                            trending_bullish_Buy_strong = (macd > signal) & (macd.shift(1) <= signal)&(macd >= 0)


                    #cross with zero line
                            Buy_cross_signal = (signal > 0) & (signal.shift(1) <= 0) #| (signal<=signal.shift()*0.05)
                            sell_cross_signal = (signal < 0) & (signal.shift(1) >= 0)


                    #reset index for idmax
                            list_dataframes = [trending_bearish_Buy_weak,trending_bullish_sell_weak,trending_bearish_sell_strong,trending_bullish_Buy_strong,Buy_cross_signal,sell_cross_signal]

                            for i in list_dataframes:
                                i.reset_index(drop=True,inplace=True)


                            positions_MACD = [trending_bearish_Buy_weak,trending_bullish_sell_weak,trending_bearish_sell_strong,trending_bullish_Buy_strong,Buy_cross_signal,sell_cross_signal]

                            n=0
                            for i in positions_MACD:
                                g=np.where(i == True)

                                if pd.DataFrame(g).empty==False:
                                    h=(np.max(g)>=list(test.index)[-3]) and (np.max(g)<=list(test.index)[-3])
                                    if h==False:
                                        n+=1
                                    if h==True:
                                        MACD_positions.iloc[j,n]=np.max(g)
                                        n+=1

                                if pd.DataFrame(g).empty==True:
                                    n+=1

                    return  MACD_positions

                    #---------------------------------------------------------------------------------------------------------------------#

            def RSI(self):
                    print('\n \n RSI ')
                    for j in range(len(self.data_pairs)):

                            if (type(self.data_pairs.at[j,'pair'])==str) | (len(self.data_pairs.at[j,'pair'])== 0):
                                continue
                            print(j,end="-")
                            test=self.data_pairs.at[j,'pair']
                    #RSI

                            RSI = ta.RSI(test['close'], timeperiod=14)

                    #positions

                            cross_overbought = (RSI < 75) & (RSI.shift(1) >= 75)
                            cross_oversold =   (RSI > 30) & (RSI.shift(1) <= 30)

                    #reset index for idmax
                            list_dataframes = [cross_overbought,cross_oversold]

                            for i in list_dataframes:
                                i.reset_index(drop=True,inplace=True)


                            positions_RSI = [cross_overbought,cross_oversold]

                            n=0
                            for i in positions_RSI:
                                g=np.where(i == True)

                                if pd.DataFrame(g).empty==False:
                                    h=(np.max(g)>=list(test.index)[-1]) and (np.max(g)<=list(test.index)[-1])
                                    if h==False:
                                        n+=1
                                    if h==True:
                                        RSI_positions.iloc[j,n]=np.max(g)
                                        n+=1
                                if pd.DataFrame(g).empty==True:
                                    n+=1

                    return  RSI_positions
    
                    #----------------------------------------------------------------------------------------------------------------#

            def BBANDS(self):
                    print('\n \n BBANDS ')
                    for j in range(len(self.data_pairs)):

                            if (type(self.data_pairs.at[j,'pair'])==str) | (len(self.data_pairs.at[j,'pair'])== 0):
                                continue
                            print(j,end="-")
                            test=self.data_pairs.at[j,'pair']
                    #Bollinger Bands


                            upper, mid, lower = ta.BBANDS(test['close'], nbdevup=2, nbdevdn=2, timeperiod=20)

                            upper = pd.Series(upper)
                            mid = pd.Series(mid)
                            lower = pd.Series(lower)

                            Percent_B = ((test['close']-lower)/(upper-lower))*100

                    #positions

                            Buy_BB =  Percent_B <= 0
                            sell_BB = Percent_B >= 100

                    #reset index for idmax
                            list_dataframes = [Buy_BB,sell_BB]

                            for i in list_dataframes:
                                i.reset_index(drop=True,inplace=True)


                            positions_BB = [Buy_BB,sell_BB]

                            n=0
                            for i in positions_BB:
                                g=np.where(i == True)

                                if pd.DataFrame(g).empty==False:
                                    h=(np.max(g)>=list(test.index)[-2]) and (np.max(g)<=list(test.index)[-2])
                                    if h==False:
                                        n+=1
                                    if h==True:
                                        BB_positions.iloc[j,n]=np.max(g)
                                        n+=1 
                                if pd.DataFrame(g).empty==True:
                                    n+=1
                    return  BB_positions

                    #----------------------------------------------------------------------------------------------------------------#            

            def OBV(self):
                    print('\n \n OBV ')
                    for j in range(len(self.data_pairs)):

                            if (type(self.data_pairs.at[j,'pair'])==str) | (len(self.data_pairs.at[j,'pair'])== 0):
                                continue
                            print(j,end="-")
                            test=self.data_pairs.at[j,'pair']
                    # On Balance Volume           
                            OBV = ta.OBV(test['close'], test['volume'])    
                            EMA60 = ta.EMA(OBV, timeperiod=60)    
                            Buy_OBV  = (OBV > EMA60) & (OBV.shift(1) <= EMA60)
                            sell_OBV = (OBV < EMA60) & (OBV.shift(1) >= EMA60)

                    #reset index for idmax
                            list_dataframes = [Buy_OBV,sell_OBV]

                            for i in list_dataframes:
                                i.reset_index(drop=True,inplace=True)


                            positions_OBV = [Buy_OBV,sell_OBV]

                            n=0
                            for i in positions_OBV:
                                g=np.where(i == True)

                                if pd.DataFrame(g).empty==False:
                                    h=(np.max(g)>=list(test.index)[-3]) and (np.max(g)<=list(test.index)[-3])
                                    if h==False:
                                        n+=1
                                    if h==True:
                                        OBV_positions.iloc[j,n]=np.max(g)
                                        n+=1 
                                if pd.DataFrame(g).empty==True:
                                    n+=1
                    return OBV_positions


class tacal(calculate_ta):
        item = {'adx':'ADX', 'wma':'WMA', 'macd':'MACD', 'rsi':'RSI', 'bbands':'BBANDS', 'obv':'OBV'}
        All = ['adx', 'wma', 'macd', 'rsi', 'bbands', 'obv']
        def get_ta(self, namepositions=All):
            index=[]
            for i in namepositions:
                index .append(f'{i}')
            pos = pd.DataFrame(columns = ['data'],index=index)    
            for i in namepositions:
                pos.at[i,'data'] =eval(f'self.{self.item[i]}()') # eval >>>> 'ADX'  ---> ADX  =  'ADX'  ---> ADX()
            return pos

       # def item_list(self, namepositions=All):
          # item=["ADX", "WMA", "MACD", "RSI", "BBANDS", "OBV"]
            #index=[]
            #for i in namepositions:
               # index .append(f'{i}')
            #pos = pd.DataFrame(columns = ['data'],index=index)    
            #for i in namepositions:
              #  pos.at[i,'data'] =eval(f'self.{self.item[i]}()') # eval >>>> 'ADX'  ---> ADX  =  'ADX'  ---> ADX()
           # return pos

                

p=tacal()
d=['adx', 'wma', 'macd', 'rsi', 'bbands', 'obv']
h=p.get_ta(d)
h


 
 ADX 
0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99-100-101-102-103-104-105-106-
 
 WMA 
0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99-100-101-102-103-104-105-106-
 
 MACD 
0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99-100-101-102-103-104-105-106-
 
 RSI 
0-1-2-3-4

,data
adx,trending_bullish trending_bearish best_pos...
wma,Buy_WMA20 SELL_WMA20 Buy_WMA60 SELL_WMA60 ...
macd,trending_bearish_Buy_weak trending_bullish...
rsi,cross_overbought cross_oversold 0 ...
bbands,Buy_BB sell_BB 0 NaN NaN 1 N...
obv,Buy_OBV sell_OBV 0 NaN NaN 1 ...


In [7]:
#h.to_pickle("h.pkl")
h.loc['rsi','data']

,cross_overbought,cross_oversold
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
102,NaN,NaN
103,NaN,NaN
104,NaN,NaN
105,NaN,NaN


In [6]:
print(pairs.iloc[6,0])
f=h.loc['rsi','data']
# pd.set_option('display.max_rows', f.shape[0]+1)
f

YFIUSDTM


,cross_overbought,cross_oversold
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
102,NaN,NaN
103,NaN,NaN
104,NaN,NaN
105,NaN,NaN


In [19]:
data_pairs = pd.read_pickle("data_pairs_kucoin_future_1d.pkl")
list(data_pairs.loc[3,'pair'].index)[-3]

397